<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/LSTM-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(LSTM(100, input_shape=(len(x_data.columns), 1), activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 41,204
Trainable params: 41,204
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_x, train_y, epochs=6, batch_size=32)

Epoch 1/6
414237/414237 [==============================] - 1549s 4ms/step - loss: 0.0406 - accuracy: 0.9862
Epoch 2/6
414237/414237 [==============================] - 1515s 4ms/step - loss: 0.0258 - accuracy: 0.9911
Epoch 3/6
414237/414237 [==============================] - 1498s 4ms/step - loss: 0.0201 - accuracy: 0.9927
Epoch 4/6
414237/414237 [==============================] - 1509s 4ms/step - loss: 0.0094 - accuracy: 0.9977
Epoch 5/6
414237/414237 [==============================] - 1495s 4ms/step - loss: 0.0070 - accuracy: 0.9985
Epoch 6/6
414237/414237 [==============================] - 1495s 4ms/step - loss: 0.0061 - accuracy: 0.9987


In [11]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 182s 2ms/step


In [12]:
confusion_matrix(test_y, pred_y)

array([[ 117727,       0,       0,       0],
       [      0,   95972,    2230,       0],
       [      0,     845, 2847065,       0],
       [      0,       0,       0,  250056]])

In [15]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000    117727
           1    0.99127   0.97729   0.98423     98202
           2    0.99922   0.99970   0.99946   2847910
           3    1.00000   1.00000   1.00000    250056

    accuracy                        0.99907   3313895
   macro avg    0.99762   0.99425   0.99592   3313895
weighted avg    0.99907   0.99907   0.99907   3313895

